# 35_Sympy_Polynomial

We are going to see how we can use `Sympy` and `Polynomial` modules to carry out symbolic calculus.

* I.    we will propagate standard errors (Sympy),
* II.   we will solve a sixth-degree equation (Polynomial),
* II.   we will solve a system of equations (Polynomial),
* IIII. we will do a Taylor expansion (Sympy).

This tutorial hopes to show how `Sympy` and `Polynomial` can carry out tedious computations for you.

In [2]:
import numpy as np
from numpy.polynomial import Polynomial
import matplotlib.pyplot as plt

In [ ]:
import math
from sympy import symbols, lambdify, diff, solve, series, simplify, Eq, exp

# I Uncertainty estimates (`Sympy`)

## Maximum uncertainty propagation

Measured quantities $x_i$ have an associated maximum uncertainty $\Delta x_i$. They can serve as input quantities of model function $f(x_1, x_2, \dots)$. The maximum uncertainties on this output quantity $y$ is

$$
\Delta y = \sum_i \left|\frac{\partial f}{\partial x_i}\right| \Delta x_i.
$$

This propagates the maximum uncertainty, which is very conservative but unrealistic as it does not allow for errors to compensate. Indeed the propagated maximum uncertainty accounts for a scenario where each measurement is done the worst way possible in terms of uncertainty and in a way they all contribute in the same direction to the uncertainty of the output quantity. The value would reach the maximum uncertainty $\Delta y$ only in the case where all the measured quantities $x_i$ would be measured with a maximum uncertainty that adds up to the output quantity, which is very very unlikely. To account for a statistical distribution of the uncertainties, it is advised to consider standard errors.

## Standard error propagation

In the case of an evenly distributed error, the standard error from all measurements should be taken as a fraction of the maximum uncertainty such that $u_{x_i} = \Delta x_i / \sqrt{3}$. For uncorrelated input quantities, the propagated standard error is

$$
u_y^2 = \sum_i \left(\frac{\partial f}{\partial x_i} u_{x_i}\right)^2.
$$


More on the topic in [french](https://ics.utc.fr/PS90/chapitre%202/co/propag_incertitudes.html), or [english](). And [the main reference from the BIPM](https://www.bipm.org/documents/20126/2071204/JCGM_100_2008_E.pdf/cb0ef43f-baa5-11cf-3f85-4dcd86f77bd6). You can see more propagations of uncertainties in `/Users/adrien/Documents/GitHub/REIN/Data_analysis/Tickle_analysis/250519_tickle_analysis.ipynb`.

# Example of propagation of standard errors ($\omega_0$ and $\alpha$)

An experiment is carried out where a single ion is observed with a camera and a PMT while a tickle drive with constant amplitude is swept in frequency over a range around the secular frequency (data from 250519). Two data points $(a_m,\sigma_m)$ and $(a_b,\sigma_b)$ are used to retrieve the secular frequency of the ion $\omega_0$ and the anharmonicity term $\alpha$ such that

$$
\omega_0 = \frac{a_m^2\omega_b - 3a_b^2\omega_m}{a_m^2 - 3 a_b^2}, \\
\alpha = \frac{8\omega_0 (\sigma_m - \sigma_b)}{3(a_m^2 - 3a_b^2)}.
$$

Once $(a_m,\sigma_m)$ and $(a_b,\sigma_b)$ measured you can compute $\omega_0$ and $\alpha$. To compute the standard error associated with those quantities you need to compute the partial derivate of those functions, for each of the four input quantities $a_m$, $a_b$, $\sigma_m$ and $\sigma_b$. While trivial, this can be quite a tedious task. Fortunately, you can use symbolic calculs with the `Sympy` module. This module can comute the partial derivates for you can give you the value for the derived standard error.

In [ ]:
# Experimental quantities
a_m = 3.640353204953884e-05
a_b = 9.395243602940294e-06

omega_m = 421016.06*2*math.pi
omega_b = 422301.20*2*math.pi

# Experimental uncertainties
Delta_a_m = 5*8e-6/11.9
Delta_a_b = 5*8e-6/11.9
Delta_omega_m = 5*8e-6/11.9
Delta_omega_b = 5*8e-6/11.9

# Output quantities
omega_0 = 422622.1341120375*2*np.pi
alpha = -5.392116209510902e+19

In [ ]:
# Symbolic variables
a_1, a_2, omega_1, omega_2 = symbols('a_1 a_2 omega_1 omega_2')
Delta_a_1, Delta_a_2, Delta_omega_1, Delta_omega_2 = symbols('Delta_a_1 Delta_a_2 Delta_omega_1 Delta_omega_2')

# Expression of omega_0 (symbolic function)
omega_0_f = (a_1**2*omega_2 - 3*a_2**2*omega_1)/(a_1**2 - 3*a_2**2)
# you have to use a different notation in the symbolic otherwise the symbolic will erase the numerical values
# so a_1 is a_m, a_2 is a_b, omega_1 is omega_m, omega_2 is omega_b

In [ ]:
# Compute the uncertainty on omega_0

# Propagation of standard errors formula
d_omega_0 = lambdify((a_1, a_2, omega_1, omega_2, Delta_a_1, Delta_a_2, Delta_omega_1, Delta_omega_2),
                    (diff(omega_0_f,a_1)*Delta_a_1)**2 + (diff(omega_0_f,a_2)*Delta_a_2)**2 + (diff(omega_0_f,omega_1)*Delta_omega_1)**2 + (diff(omega_0_f,omega_2)*Delta_omega_2)**2 )

# Numerical evaluation
d_omega_0 = math.sqrt(d_omega_0(a_m, a_b, omega_m, omega_b, 3.361344537815126e-6/math.sqrt(3), 3.361344537815126e-6/math.sqrt(3), 40/2*2*math.pi/math.sqrt(3), 40/2*2*math.pi/math.sqrt(3)))

print(d_omega_0)
print(d_omega_0/2/math.pi)

In [ ]:
# Compute the uncertainty on alpha as a exercise

# You should find a standard error of 9.112417887787878e+18

# II Solve a sixth-degree equation (`Polynomial`)

The one-dimensional dynamics of a laser-cooled single ion in a trap with a symmetric amharmonic effective trap potential along x axis is described by the following Duffing type equation

\begin{equation}
    \ddot{x} + 2\mu\dot{x} + \gamma\dot{x}^3 + \omega_0^2 x + \alpha x^3 = k_d\cos(\omega_d t + \varphi_d ).
\end{equation}

The steady-state amplitude $a$ of such oscillator is described by

\begin{equation}
    \frac{k_d^2}{4\omega_0^2} = \biggr[ \left( \sigma_d - \frac{3\alpha}{8\omega_0}a^2 \right)^2 + \left( \mu + \frac{3}{8}\gamma\omega_0^2 a^2 \right)^2\biggr]a^2.
\end{equation}

This is a sixth-order equation in $a$, where only the even terms are non-null. Thus this equation should have six solutions, but only three distinct ones. The unknown parameters are $\mu$ and $k_d$.

In [ ]:
# Function that computes the roots of the polynomial for given parameters

def find_roots(omega_z, omega_drive, cooling_rate, nl_damping_rate, B, K_eq, m_Ca):

    print('Parameters:  ')
    print(f'omega_z = {omega_z:.1f} rad/s')
    print(f'cooling_rate = {cooling_rate:.3f} rad/s')
    print(f'nl_damping_rate = {nl_damping_rate:.3f} rad s/m')
    print(f'B = {B:.3e} rad$^2$/(ms)^$2$')
    print(f'K_eq = {K_eq:.1f}')
    print(f'm_Ca = {m_Ca:.3e} kg')

    sigma = omega_drive - omega_z # drive detuning

    p_roots = np.zeros((6,len(sigma)),dtype=np.complex128) # roots will be stored here

    # loop over detunings
    for i,j in enumerate(sigma):

        # Coefficients of the polynomial
        apow6 = 9/16*(B**2 + nl_damping_rate**2*omega_z**6)
        apow4 = 3*omega_z*( (cooling_rate/2)*nl_damping_rate*omega_z**3 - j*B)
        apow2 = 4*omega_z**2*(j**2 +  (cooling_rate/2)**2)
        apow0 = -(K_eq/m_Ca)**2

        coeffs = [apow0,0,apow2,0,apow4,0,apow6]
        p = Polynomial(coeffs) # creates the polynomial object
        # p_roots.append( p.roots() )
        p_roots[:,i] = p.roots() # compute and store the roots

    return np.abs(p_roots), sigma

In [ ]:
# definition of parameters
f_z = 422500
delta_f_z = 3500
shift_f_z = -500
f_t = np.linspace(f_z+shift_f_z -delta_f_z, f_z+shift_f_z +delta_f_z, 1001)

cooling_rate = 566.43*2
nl_damping_rate = -5.92e-2
B = -5.39e19

m_Ca = 40*1.66053906660e-27
K_eq = 69429*2 * m_Ca

# Compute roots
a, sigma = find_roots(f_z*2*np.pi, f_t*2*np.pi, cooling_rate, nl_damping_rate, B, K_eq, m_Ca)

In [ ]:
# plotting the solutions

fig, ax = plt.subplots(figsize=(8,6))

ax.plot(sigma, a[0,:]*1e6, ls='-', lw = 2 , label='1st root', color='purple')
ax.plot(sigma, a[1,:]*1e6, ls='-', lw = 2 , label='2nd root', color='blue')
ax.plot(sigma, a[2,:]*1e6, ls='-', lw = 2 , label='3rd root', color='green')
ax.plot(sigma, a[3,:]*1e6, ls=':', lw = 2, label='4th root', color='yellow')
ax.plot(sigma, a[4,:]*1e6, ls=':', lw = 2, label='5th root', color='red')
ax.plot(sigma, a[5,:]*1e6, ls=':', lw = 2, label='6th root', color='xkcd:orange')

ax.set_xlabel(r'$\sigma$ (rad/s)', fontsize=14)
ax.set_ylabel('Amplitude (µm)', fontsize=14)
ax.set_title('Roots of the polynomial vs detuning', fontsize=16)
ax.legend(fontsize=12)
ax.grid()
plt.show()

# III Solve a system of two equations (`Polynomial`)

Let's consider again the steady-state amplitude $a$ of such oscillator where the following change of variable has been carried out : $\mu' = \mu + \frac{3}{8}\gamma\omega_0^2 a^2$. Once $\omega_0$ and $\alpha$ are known there are two unknown parameters $\mu'$ and $k_d$ and two experimentally determined quantities $a$ and $\sigma_d$.

\begin{equation}
    \frac{k_d^2}{4\omega_0^2} = \biggr[ \left( \sigma_d - \frac{3\alpha}{8\omega_0}a^2 \right)^2 + \left( \mu' \right)^2\biggr]a^2.
\end{equation}

This equation can be written for a set of two experimental pairs $(a_m,\sigma_m)$ and $(a_b,\sigma_b)$. This leads to a system of two equations of two unknowns that can be solved for $\mu'$ and $k_d$.

\begin{align}
    \frac{k_d^2}{4\omega_0^2} &= \biggr[ \left( \sigma_m - \frac{3\alpha}{8\omega_0}a_m^2 \right)^2 + \left( \mu' \right)^2\biggr]a_m^2 \\
     \frac{k_d^2}{4\omega_0^2} &= \biggr[ \left( \sigma_b - \frac{3\alpha}{8\omega_0}a_b^2 \right)^2 + \left( \mu' \right)^2\biggr]a_b^2 
\end{align}

In [ ]:
# Symbolic variables
gamma, k, mu = symbols('gamma k mu')

# Numerical constants
alpha = alpha
omega_0 = omega_0

# Couples (a, sigma)
a1 = a_m
sigma1 = omega_m - omega_0

a2 = a_b
sigma2 = omega_b - omega_0


# Équation 1
eq1 = Eq(
    (sigma1 - 3/8*alpha/omega_0*a1**2)**2 + (mu)**2,
    1/4*k**2/(a1**2*omega_0**2)
)

# Équation 2
eq2 = Eq(
    (sigma2 - 3/8*alpha/omega_0*a2**2)**2 + (mu)**2,
    1/4*k**2/(a2**2*omega_0**2)
)

# # Équation 3
# eq3 = Eq(
#     mu - 0.5*k/(omega_0*a1) + 3/8*gamma*omega_0**2*a1**2,
#     0
# )

# # Équation 3
# eq3 = Eq(
#     mu,
#     720/2
# )

# Résolution du système
solutions = solve((eq1, eq2), (k, mu), dict=False)
print(solutions)

# IIII Taylor expansion (`Sympy`)

We will carry out the Taylor expansion of basic functions furst, then a more complex one (Scattering rate).

In [3]:
x, k = symbols('x k')

polynome = 1 + x + x**2 + x**3 + x**7
exponential = exp(k*x)

print( simplify( series( polynome, x, 0, 5) ) )
print( simplify( series( exponential, x, 0, 5) ) )

1 + x + x**2 + x**3 + O(x**5)
1 + k*x + k**2*x**2/2 + k**3*x**3/6 + k**4*x**4/24 + O(x**5)


## Scattering rate

To model the Doppler cooling as a friction force, we need to compute the scattering rate as a function of the ion velocity. The scattering rate is given by:
$$
R_s(\mathbf{v}) = \dfrac{\Gamma_{SP}}{2}s_0 \times \frac{1}{1 + s_0 + 4\left(\frac{\delta(\mathbf{v})}{\Gamma_{SP}}\right)^2}
$$

with $\delta(\mathbf{v}) = \omega_l - \omega_0 -\mathbf{kv} = \delta_0 - \mathbf{kv}$ the laser detuning, $s_0 = I/I_{sat}$, $I_{sat} = \hbar\omega_0^3/(12\pi c^2)\Gamma_{SP}$ the saturation parameter.

We want to compute the Taylor expansion of $R_{s}$ around $v=0$ up to third order which is

\begin{align} % GitHub\REIN_Trap_Cool\Taylor_expansion.ipynb
    R_s(\mathbf{v}) = s_0& \biggl( \frac{\Gamma_{SP}}{2}r_s^{_0} \notag \\
    &+\frac{4\delta_0}{\Gamma_{SP}}r_s^{_0}{}^2 \mathbf{kv} \notag \\
    &- \frac{2}{\Gamma_{SP}}\left(1 + s_0 - 12\left(\frac{\delta_0}{\Gamma_{SP}}\right)^2\right)r_s^{_0}{}^3 \mathbf{k}^2\mathbf{v}^2 \notag \\
    &- \frac{32\delta_0}{\Gamma_{SP}^3}\left(1 + s_0 - 4\left(\frac{\delta_0}{\Gamma_{SP}}\right)^2\right)r_s^{_0}{}^4 \mathbf{k}^3\mathbf{v}^3 \notag \\
    &+ \mathcal{O}(\mathbf{v}^4) \biggr) = s_0 \sum_{i=0} K_s^{_i} r_s^{_0}{}^i \mathbf{k}^i\mathbf{v}^i.
\end{align}

In [15]:
from sympy import series

In [103]:
# Third order Taylor expansion of the scattering rate around v=0
v, k, Gamma, delta, s = symbols('v k Gamma delta s')

R_s = 0.5*Gamma * s / (1 + s + 4*((delta-k*v)/Gamma)**2)

# Compute the 4th order Taylor expansion around v=0
R_scat_taylor = series( R_s, v, 0, 4 )

In [118]:
R_s

0.5*Gamma*s/(s + 1 + 4*(delta - k*v)**2/Gamma**2)

In [119]:
R_scat_taylor

0.5*Gamma**3*s*(Gamma**2*s/(Gamma**2*s + Gamma**2 + 4*delta**2) - Gamma**2*(s + 1)/(Gamma**2*s + Gamma**2 + 4*delta**2) + Gamma**2/(Gamma**2*s + Gamma**2 + 4*delta**2) - (-Gamma**2*s/(Gamma**2*s + Gamma**2 + 4*delta**2) + Gamma**2*(s + 1)/(Gamma**2*s + Gamma**2 + 4*delta**2) - Gamma**2/(Gamma**2*s + Gamma**2 + 4*delta**2))**3 + (-Gamma**2*s/(Gamma**2*s + Gamma**2 + 4*delta**2) + Gamma**2*(s + 1)/(Gamma**2*s + Gamma**2 + 4*delta**2) - Gamma**2/(Gamma**2*s + Gamma**2 + 4*delta**2))**2 + 1)/(Gamma**2*s + Gamma**2 + 4*delta**2) + 0.5*Gamma**3*s*v*(24*delta*k*(-Gamma**2*s/(Gamma**2*s + Gamma**2 + 4*delta**2) + Gamma**2*(s + 1)/(Gamma**2*s + Gamma**2 + 4*delta**2) - Gamma**2/(Gamma**2*s + Gamma**2 + 4*delta**2))**2/(Gamma**2*s + Gamma**2 + 4*delta**2) - 16*delta*k*(-Gamma**2*s/(Gamma**2*s + Gamma**2 + 4*delta**2) + Gamma**2*(s + 1)/(Gamma**2*s + Gamma**2 + 4*delta**2) - Gamma**2/(Gamma**2*s + Gamma**2 + 4*delta**2))/(Gamma**2*s + Gamma**2 + 4*delta**2) + 8*delta*k/(Gamma**2*s + Gamma**2 + 4*

In [108]:
# Compute the first terms of the Taylor expansion manually
for n,j in enumerate(range(0,4)):
    print(diff(R_s, v, n)*(v)**n/math.factorial(n))

0.5*Gamma*s/(s + 1 + 4*(delta - k*v)**2/Gamma**2)
4.0*k*s*v*(delta - k*v)/(Gamma*(s + 1 + 4*(delta - k*v)**2/Gamma**2)**2)
-2.0*k**2*s*v**2*(1 - 16*(delta - k*v)**2/(Gamma**2*(s + 1 + 4*(delta - k*v)**2/Gamma**2)))/(Gamma*(s + 1 + 4*(delta - k*v)**2/Gamma**2)**2)
-32.0*k**3*s*v**3*(1 - 8*(delta - k*v)**2/(Gamma**2*(s + 1 + 4*(delta - k*v)**2/Gamma**2)))*(delta - k*v)/(Gamma**3*(s + 1 + 4*(delta - k*v)**2/Gamma**2)**3)


In [109]:
# Compute the first terms of the Taylor expansion manually
for n,j in enumerate(range(0,4)):
    print(simplify(diff(R_s, v, n)*(v)**n/math.factorial(n)))

0.5*Gamma**3*s/(Gamma**2*(s + 1) + 4*(delta - k*v)**2)
4.0*Gamma**3*k*s*v*(delta - k*v)/(Gamma**2*(s + 1) + 4*(delta - k*v)**2)**2
-2.0*Gamma**3*k**2*s*v**2*(Gamma**2*(s + 1) - 12*(delta - k*v)**2)/(Gamma**2*(s + 1) + 4*(delta - k*v)**2)**3
-32.0*Gamma**3*k**3*s*v**3*(delta - k*v)*(Gamma**2*(s + 1) - 4*(delta - k*v)**2)/(Gamma**2*(s + 1) + 4*(delta - k*v)**2)**4


In [112]:
# Evaluate the first terms numerically
for n,j in enumerate(range(0,4)):
    temp_order = lambdify( (v, k, Gamma, delta, s), diff(R_s, v, n)*(v)**n/math.factorial(n) )
    print(temp_order(10, 2*math.pi/(369.5e-9), 2*math.pi*21.7e6, -8.3*21.7*2*np.pi*1e6, 8))

1459821.8751930953
-372212.2215255368
70591.8154439032
-11800.137278703136
